<a href="https://colab.research.google.com/github/tristantoupin/ECSE415-FinalProject/blob/master/SVM_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifer

In [1]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import cv2
import matplotlib.pyplot as plt
import os
import math
import time

from skimage.feature import hog

### Determine whether to use the dataset from Google Cloud storage or local storage

In [2]:
# Change this value depending whether you're using a cloud or local environment
is_using_cloud = False

if is_using_cloud:
    import tarfile
    from google.colab import drive
    drive.mount('/content/gdrive')
    
    path_classification = '/content/gdrive/My Drive/Colab Notebooks/Project/MIO-TCD-Classification.tar'
    tar = tarfile.open(path_classification)
else:
    path_classification = './MIO-TCD-Classification'

### Helper functions for image plotting, importing, and preprocessing

In [3]:
def plot_images(list_of_images, max_col = 4):
    """
    @brief Plot a list of given images with
    @param list_of_images List of images to plot
    @param max_col Number of colomns to plot your figures
    """
    n = len(list_of_images)
    if n == 1:
        plt.imshow(list_of_images[0]); plt.axis('off'); plt.show()
    else:
        # get number of columns and rows required
        r, c = 1, n
        if n > max_col:
            c = max_col
            r = int(math.ceil(n/max_col))
    
        fig = plt.figure(figsize=(17, max_col * r))
        for i, (img) in enumerate(list_of_images):
            ax = fig.add_subplot(r, c, (i+1))
            ax.set_title("Image " + str(i))
            ax.axis('off')
            ax.imshow(img, cmap=plt.cm.gray)

In [4]:
def pad_resize_images(img_list, output_size):
    """
    @brief Resize each image within a list to a given shape while also keeping its aspect ratio by the use of padding
    @param img_list The list of images to resize
    @param output_size The shape of the output images are output_size x output_size
    """
    BLACK = 0
    result = np.empty_like(img_list)
    
    for i, img in enumerate(img_list):
        
        height, width = img.shape
        ratio = float(output_size) / max([height, width])
        height_new, width_new = tuple([int(val * ratio) for val in (height, width)])
        img_resized = cv2.resize(img, (height_new, width_new))
        
        height_adjust = output_size - height_new
        width_adjust = output_size - width_new
        
        top = math.ceil(height_adjust / 2)
        bot = height_adjust - top
        left = math.ceil(width_adjust / 2)
        right = width_adjust - left
        
        result[i] = cv2.resize(
            cv2.copyMakeBorder(img_resized, top, bot, left, right, cv2.BORDER_CONSTANT, value=BLACK),
            (output_size, output_size))
    return result

In [5]:
def performance_random_clf(x_tr, y_tr, x_test, y_test, r = 1234):
    clf = DummyClassifier(strategy = 'uniform', random_state = r)
    clf.fit(x_tr, y_tr)
    preds = clf.predict(x_test)
    metric = metrics.classification_report(y_test, preds)
    return metric
    
def performance_majority_classifier(x_tr, y_tr, x_test, y_test, r = 1234):
    most_common_val = stats.mode(y_tr).mode[0]
    preds = np.full((y_test.shape), most_common_val)
    metric = metrics.classification_report(y_test, preds)
    return metric

def fine_tune_svm(x_tr, y_tr, folds=10):
  # Set the parameters by cross-validation
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100]}]
    models = []
    clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=folds,
                       scoring = 'accuracy', n_jobs = -1, verbose = 1)
    clf.fit(x_tr, y_tr)
    models.append(clf)
    print(clf.best_params_)

    return models

def compute_svm_predictions(x_train, y_train, x_val):
    model = svm.SVC()
    model.fit(x_train, y_train)
    preds = model.predict(x_val)
    return preds

#### Import a subset of or all images for each category with their relevant label

In [6]:
path_classification_train = os.path.join(path_classification, 'train')

#
# Change this number if you want more or less images. Set it to -1 if you want all images
# NOTE: This will ATTEMPT to have the same number of images per category. The categories
# have a large difference in number of images.
#
number_images_category = 5000
categories = os.listdir(path_classification_train)
classification_images = []
y_tr = []

start_time = time.time()
for category in categories:
    path_category = os.path.join(path_classification_train, category)
    for i, image_name in enumerate(os.listdir(path_category)):
        
        if i == number_images_category:
            break
            
        image = cv2.imread(os.path.join(path_category, image_name), cv2.IMREAD_GRAYSCALE)
        classification_images.append(image)
        
        label = categories.index(category)
        y_tr.append(label)

classification_images = np.array(classification_images)
y_tr = np.array(y_tr)

assert len(classification_images) == len(y_tr)

print('Elapsed Time:', time.time() - start_time)
print('Number of Images:', len(classification_images))

Elapsed Time: 18.414430618286133
Number of Images: 46017


In [7]:
classification_images, y_tr = shuffle(classification_images, y_tr)

#### Import a number of test images

In [8]:
path_classification_test = os.path.join(path_classification, 'test')

#
# Change this number if you want more or less images. Set it to -1 if you want all images
#
number_images_test = 0
images_test = []

for i, image_name in enumerate(os.listdir(path_classification_test)):
    if i == number_images_test:
        break
    
    image = cv2.imread(os.path.join(path_classification_test, image_name), cv2.IMREAD_GRAYSCALE)
    images_test.append(image)
    
images_test = np.array(images_test)
assert len(images_test) == number_images_test

#### Extract HoG features from training images

In [15]:
hog_images = []


for image in classification_images[:1000]:

    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=False)
    
    hog_images.append(hog_image)

C:\ProgramData\Anaconda3\envs\opencv-env\lib\site-packages\skimage\feature\_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)


#### Pad images to maintain one common aspect ratio

In [17]:
size = 128
padded_images_train = pad_resize_images(classification_images, size)
padded_images_hog = pad_resize_images(hog_images, size)

#### Flatten images to prepare for SVM

In [18]:
x_train = np.array([x.flatten() for x in padded_images_train])
x_train_hog = np.array([x.flatten() for x in padded_images_hog])

In [19]:
svm_model = fine_tune_svm(x_train[:100], y_tr[:100], folds=10)
svm_model_hog = fine_tune_svm(x_train_hog[:100], y_tr[:100], folds=10)

Fitting 10 folds for each of 11 candidates, totalling 110 fits


C:\ProgramData\Anaconda3\envs\opencv-env\lib\site-packages\sklearn\model_selection\_split.py:626: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 110 out of 110 | elapsed:   14.8s finished
C:\ProgramData\Anaconda3\envs\opencv-env\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'C': 1, 'kernel': 'linear'}
Fitting 10 folds for each of 11 candidates, totalling 110 fits


C:\ProgramData\Anaconda3\envs\opencv-env\lib\site-packages\sklearn\model_selection\_split.py:626: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 110 out of 110 | elapsed:   14.7s finished
C:\ProgramData\Anaconda3\envs\opencv-env\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'C': 1, 'kernel': 'linear'}


In [24]:
best_svm_model = svm_model[0]
best_svm_model_hog = svm_model_hog[0]

In [34]:
model_predictions = best_svm_model.predict(x_train[300:400])
model_hog_predictions = best_svm_model_hog.predict(x_train[300:400]) 

results = []
results_hog = []

for i, category in enumerate(y_tr[300:400]):
    results.append(model_predictions[i] == category)
    results_hog.append(model_hog_predictions[i] == category)

print('Non HoG Model Correct Predictions:', np.count_nonzero(results))
print('HoG Model Correct Predictions:', np.count_nonzero(results_hog))

Non HoG Model Correct Predictions: 38
HoG Model Correct Predictions: 3


#### Train and fine tune SVM model

In [15]:
start_time= time.time()
best_svm_model = fine_tune_svm(x_tr[:100], y_tr[:100], folds=10)
print(best_svm_model)
print('Elapsed Time', time.time() - start_time)

Fitting 10 folds for each of 11 candidates, totalling 110 fits


C:\ProgramData\Anaconda3\envs\opencv-env\lib\site-packages\sklearn\model_selection\_split.py:626: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 110 out of 110 | elapsed:   14.1s finished
C:\ProgramData\Anaconda3\envs\opencv-env\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'C': 1, 'kernel': 'linear'}
[GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'kernel': ['rbf'], 'gamma': [0.001, 0.0001], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)]
Elapsed Time 14.698078155517578


In [18]:
best_model = svm.SVC(C = 1, gamma = 0.001, kernel = 'linear')
best_model.fit(x_tr[:100], y_tr[:100])

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [23]:
preds = best_model.predict(x_tr[100:200])

[ 1  7  1 10  4 10  4 10 10  4  5  9  7  4  1  1  1  1  9  0  1  1  9  4
  3  4  5  1  7  5  2  9  5  1  1  9  1  7  7  5  1  2  2  1  1  4  4  1
  7  1  1  8  2  1  5  7  2  9  1  2  4  1  2  1  1  1  8  2  1  1  3  1
  1  7  5 10  7  1  4  1  5 10  7  4  2  1  4  7  1  4  4  2  4  1  0  1
  4  2  2  5]
